In [56]:
import pandas as pd
PATH = 'epir_train/'
articles = pd.read_csv(PATH + 'articles.csv', index_col='Unnamed: 0')
life_situations = pd.read_csv(PATH + 'life_situations.csv', index_col='Unnamed: 0')
news = pd.read_csv(PATH + 'news.csv', index_col='Unnamed: 0')
services = pd.read_csv(PATH + 'services.csv', index_col='Unnamed: 0')

In [57]:
def clean_df(df):
    text_cols = []
    for col in df.columns:
        if col not in ['id', 'sys_lang', 'subid', 'URL']:
            text_cols.append(col)

    df[text_cols] = df[text_cols].astype(str)
    mask = df[text_cols].apply(lambda x: x.str.split().str.len() < 10).all(axis=1)

    print("Rows that will be deleted:")
    print(df[mask])

    df = df.drop(df[mask].index)
    return df

In [58]:
# apply cleaning for all dfs
articles = clean_df(articles)
life_situations = clean_df(life_situations)
news = clean_df(news)
services = clean_df(services)

Rows that will be deleted:
            id sys_lang                           projects  \
2892     81739       kk                               test   
2900    103237       qq                      almaty-almaly   
2906    113705       kk                             atyrau   
2908     27055       qq                                mti   
2921      7844       ru                              enbek   
...        ...      ...                                ...   
102834   17016       kk                        vko-oskemen   
102852  113731       kk                             atyrau   
102853   54996       qq                        vko-kurchum   
102862   38778       qq  kostanai-altynsarin-audany-akimat   
102878  124529       kk                  karaganda-nurinsk   

                                                    title  \
2892                                                 тест   
2900    Áleýmettik meditsinalyq saqtandyrý máseleleri ...   
2906                                         

In [59]:
# list the column names for all dataframes
print("Articles columns:")
print(articles.columns)
print("Life situations columns:")
print(life_situations.columns)
print("News columns:")
print(news.columns)
print("Services columns:")
print(services.columns)

Articles columns:
Index(['id', 'sys_lang', 'projects', 'title', 'content', 'url'], dtype='object')
Life situations columns:
Index(['id', 'sys_lang', 'intro', 'title_main', 'title_sub', 'subid',
       'instruction', 'URL'],
      dtype='object')
News columns:
Index(['id', 'sys_lang', 'projects', 'title', 'body', 'short_description',
       'url'],
      dtype='object')
Services columns:
Index(['id', 'sys_lang', 'additional_info', 'description', 'full_title',
       'short_title', 'title', 'result_description', 'url'],
      dtype='object')


In [62]:
import re

def select_translation(group, big_text_column):
    # Try to find English translations
    en_translations = group[group['sys_lang'] == 'en']
    if not en_translations.empty:
        # If there are multiple English translations, select the one with the most Latin letters
        return en_translations.loc[
            en_translations[big_text_column].apply(
                lambda x: len(re.findall('[a-zA-Z]', x))
                ).idxmax()
        ]
    # If no English translations, try to find Russian translations
    ru_translations = group[group['sys_lang'] == 'ru']
    if not ru_translations.empty:
        return ru_translations.iloc[0]
    # If no Russian translations, try to find Kazakh translations
    kz_translations = group[group['sys_lang'] == 'kz']
    if not kz_translations.empty:
        return kz_translations.iloc[0]
    # If no English, Russian or Kazakh translations, select one of the available translations
    return group.iloc[0]

def select_translations(df, big_text_column):
    return df.groupby('id').apply(select_translation, big_text_column).reset_index(drop=True)

In [63]:
selected_articles = select_translations(articles, 'content')
selected_articles

,id,sys_lang,projects,title,content,url
0,1,en,mam,О защите детей от информации,2 июля 2018 года Главой государства подписан З...,https://www.gov.kz/memleket/entities/mam/press...
1,2,ru,mam,Положение о министерстве информации и комуникц...,Положение о министерстве Министерство информац...,https://www.gov.kz/memleket/entities/mam/press...
2,13,kk,mam,Қазақстан Республикасы Ақпарат және коммуникац...,Қазақстан Республикасы Ақпарат және коммуникац...,https://www.gov.kz/memleket/entities/mam/press...
3,17,ru,mam,Контакты Департамента Цифровизации,Департамент цифровизации 785 Директор Бажаева ...,https://www.gov.kz/memleket/entities/mam/press...
4,20,kk,mam,Халықаралық ынтымақтастық департаментінің байл...,Халықаралық ынтымақтастық департаменті 757 Дир...,https://www.gov.kz/memleket/entities/mam/press...
...,...,...,...,...,...,...
68128,136238,ru,anticorruption,ГРАФИК проведения собеседования,"№ Должность Фамилия, имя, отчество (при его на...",https://www.gov.kz/memleket/entities/anticorru...
68129,136239,qq,almobl-uigur,Dúniezhúzilik Banktiń ókili Himanshi Dzhein t...,"2023 zhylǵy 28 qyrkúiekte ""Mindetti zhinaqtaýs...",https://www.gov.kz/memleket/entities/almobl-ui...
68130,136241,qq,departament-kkbtu-turkestan,Qyzylshanyń aldyn alý. Vaktsinatsiia infektsii...,Qyzylsha aýrýy - kóbine balalar arasynda zhii ...,https://www.gov.kz/memleket/entities/departame...
68131,136243,ru,almobl-uigur,За 8 месяцев чистый инвестиционный доход вклад...,"Чистый инвестиционный доход, начисленный на сч...",https://www.gov.kz/memleket/entities/almobl-ui...


In [65]:
selected_news = select_translations(news, 'body')
selected_news

,id,sys_lang,projects,title,body,short_description,url
0,36,ru,mam,Телеграммы и письма по случаю национального пр...,Председатель Китайской Народной Республики Си ...,nan,https://www.gov.kz/memleket/entities/mam/press...
1,271,kk,mam,Astana Hub алғашқы акселераттау бағдарламасын ...,Astana Hub IT-стартаптары халықаралық технопар...,nan,https://www.gov.kz/memleket/entities/mam/press...
2,281,en,mam,Abayev attended the North Kazakhstan Akim’s re...,Minister of Information and Communications att...,nan,https://www.gov.kz/memleket/entities/mam/press...
3,285,en,mam,Hearing of reports on implementation of the Na...,"Headed by Dauren Abayev, Minister of Informati...",nan,https://www.gov.kz/memleket/entities/mam/press...
4,287,en,mam,The Journalists Contest: “European Union – Kaz...,To celebrate the 25th anniversary of diplomati...,nan,https://www.gov.kz/memleket/entities/mam/press...
...,...,...,...,...,...,...,...
90618,631457,ru,karaganda,«Взгляд сквозь время»: В Темиртауском музее пр...,В Темиртауском городском историко-краеведческо...,nan,https://www.gov.kz/memleket/entities/karaganda...
90619,631460,qq,almaty,Aldaǵy bes zhylda Almaty aglomeratsiiasynda ne...,"Qolzhetimdi turǵyn úi, zhańa áleýmettik nysand...",nan,https://www.gov.kz/memleket/entities/almaty/pr...
90620,631463,kk,almaty,Алматыдағы Құлжа трактіндегі жаңа жолайрығы ав...,Алматыда Бухтарминская көшесі мен Құлжа тракті...,nan,https://www.gov.kz/memleket/entities/almaty/pr...
90621,631485,qq,pavlodar-din,Q.K. Toqaev mektepke hidzhab kiip barý máseles...,Qasym-Zhomart Toqaev Respýblikalyq pedagogtar ...,nan,https://www.gov.kz/memleket/entities/pavlodar-...


In [66]:
selected_life_situations = select_translations(life_situations, 'instruction')
selected_life_situations

,id,sys_lang,intro,title_main,title_sub,subid,instruction,URL
0,1,en,"During the preparation for childbirth, you sho...",You are expecting a child,Zhúktilik kezinde dekret demalysyna shyǵý zhán...,3,Dekrettik demalys Zhumys isteitin áielderge bo...,https://beta2.egov.kz/situations/1/3?lang=en
1,2,en,"Before travelling to another country, it is ne...",You're going on a vacation abroad,How to prepare for a stay in a foreign country,11,In order for pastime in another country to be ...,https://beta2.egov.kz/situations/2/11?lang=en
2,3,en,"You, as a citizen of Kazakhstan, have the righ...",Receiving a land plot for IHC,Provision of land plot,14,As soon as it is your turn to receive a land p...,https://beta2.egov.kz/situations/3/14?lang=en
3,4,en,"When you decide to marry your loved one, you n...",Creating a family,How to apply online,18,You can register your marriage online through ...,https://beta2.egov.kz/situations/4/18?lang=en
4,5,en,Are you preparing to go on a well-deserved vac...,You are going to retire,Amount of pension payments and their calculation,26,Let's find out what types of pension payments ...,https://beta2.egov.kz/situations/5/26?lang=en
...,...,...,...,...,...,...,...,...
81,315,ru,Родители в декрете имеют право получать социал...,Как рассчитать декретные,How is the monthly childcare social allowance ...,809,According to subparagraph 2) of paragraph 1 of...,https://beta2.egov.kz/situations/315/809?lang=ru
82,316,ru,Дети и подростки теперь проводят больше времен...,Как предотвратить кибербуллинг,Tips for parents to keep their children safe,811,Teach your children about cyberbullying. By kn...,https://beta2.egov.kz/situations/316/811?lang=ru
83,317,ru,Индивидуальные предприниматели или юридические...,Кто должен применять POS-терминалы,Are POS-terminals subject to registration with...,815,This equipment is not registered with the stat...,https://beta2.egov.kz/situations/317/815?lang=ru
84,318,ru,При сокращении численности или штата работнико...,Что нужно знать работодателю при сокращении шт...,Соблюдение запретов при сокращении,818,"Для начала необходимо выяснить, попадает ли ра...",https://beta2.egov.kz/situations/318/818?lang=ru


In [67]:
selected_services = select_translations(services, 'result_description')
selected_services

,id,sys_lang,additional_info,description,full_title,short_title,title,result_description,url
0,2999,en,Service standard,Who is civil officer? It is a citizen of the R...,Enrollment to candidate pool of administrative...,Enrollment to candidate pool of administrative...,Enrollment to candidate pool of administrative...,Extract from the order on enrollment to candid...,https://beta2.egov.kz/services/2999?lang=en
1,3000,en,Rules for providing state services,Dear citizens of the Republic of Kazakhstan! I...,Assignment of state social allowance for disab...,Assignment of state social allowance for disab...,Assignment of state social allowance for disab...,Notice of appointment (refusal in appointment)...,https://beta2.egov.kz/services/3000?lang=en
2,3002,en,Service standard,Information is being updated. What is pension ...,Issuance of information on receipt and flow of...,Issuance of abstract on pension contributions,Certificate of pension contributions,Obtaining a certificate on pension contributio...,https://beta2.egov.kz/services/3002?lang=en
3,3003,en,nan,Information is being updated. The service is a...,Assignment of state social allowance for disab...,Obtaining an information on the assignment of ...,Obtaining an information on the assignment of ...,Certificate with informational content on assi...,https://beta2.egov.kz/services/3003?lang=en
4,3004,en,Rules for providing State service,Information is being updated. The public servi...,Issuance of documents on advanced training and...,Issuance of documents on advanced training and...,Issuance of documents on advanced training and...,On completion of the qualification: certificat...,https://beta2.egov.kz/services/3004?lang=en
...,...,...,...,...,...,...,...,...,...
882,4553,ru,nan,nan,Выдача архивных справок и/или копий архивных д...,Выдача архивных справок и/или копий архивных д...,Выдача архивных справок и/или копий архивных д...,nan,https://beta2.egov.kz/services/4553?lang=ru
883,4561,ru,nan,nan,Корректировка ошибочных сведений объектов недв...,Корректировка ошибочных сведений объектов недв...,Корректировка ошибочных сведений объектов недв...,nan,https://beta2.egov.kz/services/4561?lang=ru
884,4562,ru,nan,nan,Выдача свидетельства на переоборудование автот...,Выдача свидетельства на переоборудование автот...,Выдача свидетельства на переоборудование автот...,nan,https://beta2.egov.kz/services/4562?lang=ru
885,4565,ru,nan,nan,Предоставление гарантий по кредитам в рамках Г...,Предоставление гарантий по кредитам в рамках Г...,Предоставление гарантий по кредитам в рамках Г...,nan,https://beta2.egov.kz/services/4565?lang=ru
